In [1]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
import os
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import TextLoader

c:\Python311\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
pinecone.init(api_key=os.environ['OPENAI_API_KEY'])
index_name = "coursecrafter"
index = pinecone.Index(index_name)

retriever = index.as_retriever()


llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages= True)
chain = ConversationalRetrievalChain.from_llm(llm, retriever= retriever, memory= memory)

AttributeError: 'Index' object has no attribute 'as_retriever'

In [ ]:
query = 'which professor has the lowest gpa'
chain.run({'question': query})